**Learning outcomes**

* Define a Hamiltonian and explain what it means with respect to a quantum system.
* Compute unitary evolution for a simple, single-qubit physical system.

In [7]:
import numpy as np
import pennylane as qml

More specifically, the unitaries $U$ evolving a quantum system arise from a special operator $\hat{H}$ called the **Hamiltonian**, which measures the system's energy. According to **Schrödinger's equation**, for a system with Hamiltonian $\hat{H}$, the corresponding unitary evolving it for a time $t$ is
$$U = e^{-i\hat{H}/\hbar}$$
where $\hbar \approx 10^{34} J*S$ is Planck's (reduced) constant.

When we say the little magnet "likes to align", what we really mean is the energy of the system is lower when the little magnet is aligned. This is captured by the Hamiltonian:
![equation](./images/H.3.1.1.png)
, where $Z$ is a Pauli $Z$, $e = 1.6$ x $10^{-19} C$ is the charge of electron, and $m_e = 9.1$ x {10^31} kg its mass. We've lumped some of the constants into $\alpha = eB/2m_e$ for convenience.

More general:
![equation](./images/H.3.1.2.png)

In [6]:
def unitary_check(operator):
    """Checks if a complex-valued matrix is unitary.

    Args:
        operator (array[complex]): A square complex-valued array.

    Returns:
        bool: Whether the matrix is unitary or not.
    """
    adjoint = np.transpose(np.conjugate(operator))
    check = False

    #operator @ adjoint
    left = np.matmul(operator,adjoint)
    check =  np.allclose(left, np.identity(len(operator)))

    right = np.matmul(adjoint,operator)
    check = np.allclose(right, np.identity(len(operator)))

    return check

**Codercise H.3.1.** (a) Complete the code to build the unitary above. We can verify the output is unitary using *unitary_check*.
* $e^{i\alpha t Z}\Ket{0} = e^{i\alpha t}\Ket{0} $
* $e^{i\alpha t Z}\Ket{1} = e^{-i\alpha t}\Ket{1} $

In [15]:
def mag_z_unitary(B, time):
    """Creates a unitary operator to evolve the state of an electron in
    a magnetic field.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time (t) we evolve the electron state for.

    Returns:
        array[complex]: The unitary matrix implementing time evolution.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)

    matrix = np.array([[0, 0j], [0, 0j]]) # CHANGE THIS

   # matrix[0,0] = math.cos(alpha * t) + (1j * math.sin(alpha*t))
   # matrix[1,1] = math.cos(alpha*t) - (1j * math.sin(alpha*t))

    matrix[0][0] = np.exp(1j * alpha * t)
    matrix[1][1] = np.exp(-1j * alpha * t)
    return matrix

B, t = 0.1, 0.6
if unitary_check(mag_z_unitary(B, t)):
    print("The output is unitary for B =", B, "and t =", t, ".")

The output is unitary for B = 0.1 and t = 0.6 .


(b) Write a circuit which evolves the bar magnet in a magnetic field for time $t$, starting in state $\Ket{0}$ and applying the unitary you constructed in the previous exercise. The `mag_z_unitary` function is available for you to use.

In [17]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def mag_z_0_v1(B, time):
    """Simulates an electron (initial state |0>) in a magnetic field, using a
    unitary matrix.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron state for.

    Returns:
        array[complex]: The state of the system after evolution.
    """
    qml.QubitUnitary(mag_z_unitary(B,time), wires=0)

    return qml.state()

(c) Rewrite the circuit from part (b) using PennyLane's `qml.RZ` method, and check it gives the same state for a given pair of parameter values.

Tip. Recall that $R_{z}(\phi) = e^{-i\phi Z/2}$, so $e^{i\alpha tZ} = R_{z}(-2\alpha t) $.

In [18]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def mag_z_0_v2(B, time):
    """Simulates an electron (initial state |0>) in a magnetic field, using a
    Z rotation.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron state for.

    Returns:
        array[complex]: The state of the system after evolution.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)

    qml.RZ(-2 * alpha * t, wires=0)
    return qml.state()

B, t = 0.1, 0.6
if np.allclose(mag_z_0_v1(B, t), mag_z_0_v2(B, t)):
    print("The two circuits give the same answer!")

The two circuits give the same answer!


So, we've seen how to take something physical, turns its evolution into a mathematical expression, and then to do the same thing on a quantum computer!

(d) Hitting the submit button will plot the real and imaginary parts of $\Bra{\psi(t)}\Ket{0}$ separately, where $\Ket{\psi(t)} = e^{i\alpha tZ}\Ket{0}$. What do you think is happening to the state of the bar magnet as it evolves over time?

![plot1](./images/H.3.1.3.png)
![plot2](./images/H.3.1.4.png)


The real and imaginary parts are varying sinusoidally, and out of phase. This suggests that $\Bra{\psi(t)}\Ket{0} = e^{i\Theta t}$ is simply rotating around the unit circle in the complex plane, for some parameter $\Theta$. In other words, the amplitude for $\Ket{0}$ simply picks up a phase as time evolves.


(e) Finally, write a circuit (using the $Z$ rotation) which evolves a qubit starting in state $\Ket{+}$. From the plots of $\Braket{X}$ and $\Braket{Y}$, what do you think is happening?

In [19]:
dev = qml.device("default.qubit", wires=1)

def evolve_plus_state(B, time):
    """Simulates an electron (initial state |+>) in a magnetic field.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron state for.
    """
    e = 1.6e-19
    m_e = 9.1e-31
    alpha = B*e/(2*m_e)

    qml.Hadamard(wires=0)
    qml.RZ(-2 * alpha * time, wires=0)

@qml.qnode(dev)
def mag_z_plus_X(B, time):
    """Simulates an electron (initial state |+>) in a magnetic field and returns <X>.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron state for.

    Returns:
        float: Expectation value for the Pauli X operator.
    """
    evolve_plus_state(B, time)
    return qml.expval(qml.PauliX(0))

@qml.qnode(dev)
def mag_z_plus_Y(B, time):
    """Simulates an electron (initial state |+>) in a magnetic field and returns <Y>.

    Args:
        B (float): The strength of the field, assumed to point in the z direction.
        time (float): The time we evolve the electron state for.

    Returns:
        float: Expectation value for the Pauli X operator.
    """
    evolve_plus_state(B, time)
    return qml.expval(qml.PauliY(0))

![X - measure](./images/H.3.1.5.png)
![Y - measure](./images/H.3.1.6.png)

The plots of $\Braket{X}$ and $\Braket{Y}$ are oscillating and out of of phase, just like the real and imaginary components of $\Bra{\psi(t)}\Ket{0}$. This suggests that they too are components of a vector on a circle, but now this is the spin vector itself rather than a complex number. We picture this below:
![spin outcome](./images/H.3.1.7.png)


In general, the spin vector will rotate around the $z$-axis in a mation called *Larmor precession*